# Import Libraries and Code for Plotting

In [1]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import cupy as cp

In [2]:
VariantNames = ['Base','FixedRandomSample','ProportionalRandomSample','Agglomerative','RandomParameters']
ValidFunctions = list(range(1,10))+list(range(20,28))

In [ ]:
def Table_MediansIterations(Experiment,DimensionNumber,VariantNames,Tolerance):
    # Firt median of optimal values, then search iteration
    data_medians_iterations = dict()

    for number_function in ValidFunctions:
        iterations_variants = list(IterationsVariants_MediansIterations(Experiment,DimensionNumber,VariantNames,number_function,Tolerance))
        data_medians_iterations[f"F{number_function}"] = iterations_variants

    return data_medians_iterations

def IterationsVariants_MediansIterations(Experiment,DimensionNumber,VariantNames,NumberFunction,Tolerance):
    optimal_value = cp.array(-(NumberFunction*cp.float64(100)+Tolerance),dtype=float)

    for variant_name in VariantNames:
        data_optimal_values = cp.loadtxt(f"{Experiment[0]}_{Experiment[1]}_Dim_{DimensionNumber}/{variant_name}/OptimalValues_F{NumberFunction}.csv",dtype=float,delimiter=',')
        medians_optimal = -cp.median(data_optimal_values,axis=0)

        yield medians_optimal.searchsorted(optimal_value).get()

In [63]:
def Table_IterationsMedians(Experiment,DimensionNumber,VariantNames,Tolerance):
    # First search iteration to reach optimal, then median of iterations
    data_iterations_meadians = dict()

    for number_function in ValidFunctions:
        iterations_variants = list(IterationsVariants_IterationsMedians(Experiment,DimensionNumber,VariantNames,number_function,Tolerance))
        data_iterations_meadians[f"F{number_function}"] = iterations_variants

    return data_iterations_meadians

def IterationsVariants_IterationsMedians(Experiment,DimensionNumber,VariantNames,NumberFunction,Tolerance):
    tolerance = -cp.array(Tolerance,dtype=float)
    
    for variant_name in VariantNames:
        data_optimal_values = cp.loadtxt(f"{Experiment[0]}_{Experiment[1]}_Dim_{DimensionNumber}/{variant_name}/OptimalValues_F{NumberFunction}.csv",dtype=float,delimiter=',')
        data_optimal_values -= NumberFunction*cp.float64(100)
        iterations_optimal = cp.apply_along_axis(lambda arr : cp.searchsorted(-arr,tolerance),axis=1,arr=data_optimal_values)

        yield cp.median(iterations_optimal).get()

# Results

In [67]:
globals_results = globals()

## Using Optimal Values First

In [ ]:
type_result = 'Medians'
dimension_number = '2'
tolerance = 0.1

In [ ]:
experiment = ['02','100']
variant_names = VariantNames

globals_results[f'{type_result}Results_{experiment[1]}'] = Table_MediansIterations(experiment,dimension_number,variant_names,tolerance)

In [ ]:
variant_names = [VariantNames[0],VariantNames[-1]]

for population_size in ['12','25','37','50','62','75','87']:
    experiment[1] = population_size
    globals_results[f'{type_result}Results_{experiment[1]}'] = Table_MediansIterations(experiment,dimension_number,variant_names,tolerance)

## Using Iterations Firts

In [ ]:
type_result = 'Iterations'
dimension_number = '2'
tolerance = 0.1

In [ ]:
experiment = ['02','100']
variant_names = VariantNames

globals_results[f'{type_result}Results_{experiment[1]}'] = Table_IterationsMedians(experiment,dimension_number,variant_names,tolerance)

In [ ]:
variant_names = [VariantNames[0],VariantNames[-1]]

for population_size in ['12','25','37','50','62','75','87']:
    experiment[1] = population_size
    globals_results[f'{type_result}Results_{experiment[1]}'] = Table_IterationsMedians(experiment,dimension_number,variant_names,tolerance)